In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

##YOLO 環境建置 

In [ ]:
# clone
! cd /content/drive/MyDrive/yolo_file; git clone https://github.com/AlexeyAB/darknet.git

# 須將資料夾放置於GoogleDrive: yolo_file
# 調整內容
! sed -i "s/GPU=0/GPU=1/g" /content/drive/MyDrive/yolo_file/darknet-master/Makefile
! sed -i "s/CUDNN=0/CUDNN=1/g" /content/drive/MyDrive/yolo_file/darknet-master/Makefile
! sed -i "s/OPENCV=0/OPENCV=1/g" /content/drive/MyDrive/yolo_file/darknet-master/Makefile

# 環境建置
! cd /content/drive/MyDrive/yolo_file/darknet-master; make

In [ ]:
# 下載預訓練模型(使用yolov4.conv.137)
! cd /content/drive/MyDrive/yolo_file/darknet-master; wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
! cd /content/drive/MyDrive/yolo_file/darknet-master; wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

## YOLO 訓練資料準備

In [ ]:
# 圖檔標籤準備 (主辦單位提供之標註) (需自行修改位置)
df = pd.read_csv(r'/content/drive/MyDrive/t_brain/Adam/public_training_data_adam.csv')
# 取最小
df['xmin'] = df[['top right x','top left x','bottom right x','bottom left x']].min(axis=1)
df['ymin'] = df[['top right y','top left y','bottom right y','bottom left y']].min(axis=1)
df['xmax'] = df[['top right x','top left x','bottom right x','bottom left x']].max(axis=1)
df['ymax'] = df[['top right y','top left y','bottom right y','bottom left y']].max(axis=1)
# 產出 yolo training 所需之 x,y,w,h
df = df.assign(x = lambda d : (d['xmin'] + (d['xmax']-d['xmin'])/2) *1.0 / (1232))
df = df.assign(y = lambda d : (d['ymin'] + (d['ymax']-d['ymin'])/2) *1.0 / (1028))
df = df.assign(w = lambda d : (d['xmax']-d['xmin'])*1.0 / (1232))
df = df.assign(h = lambda d : (d['ymax']-d['ymin'])*1.0 / (1028))

In [ ]:
# txt檔案,寫入label (放置於與圖檔相同之位置) (yolov4 training 所需)
for _,x in df.iterrows():
    path = f"/content/drive/MyDrive/t_brain/public_training_data/{x['filename']}.txt"
    with open(path, 'w') as f:
        f.write(f"0 {x['x']} {x['y']} {x['w']} {x['h']}")

In [ ]:
# 訓練測試 80%/20% 切分
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# 產製 train.txt & test.txt 供YOLOv4取用 (紀錄訓練時讀圖檔的位置)

# 準備train.txt
path = f"/content/drive/MyDrive/yolo_file/darknet-master/cfg_steel/train.txt"
with open(path, 'w') as f:
    for _,x in X_train.iterrows():
        f.write(f"/content/drive/MyDrive/t_brain/public_training_data/{x['filename']}.jpg\n")
        
# 準備test.txt       
path = f"/content/drive/MyDrive/yolo_file/darknet-master/cfg_steel/test.txt"
with open(path, 'w') as f:
    for _,x in X_test.iterrows():
        f.write(f"/content/drive/MyDrive/t_brain/public_training_data/{x['filename']}.jpg\n")

## YOLO Training

In [ ]:
# training yolov4 subdivision調整為64(記憶體)
# 需準備訓練設定檔 obj.data / yolov4_ad.cfg
!chmod +x /content/drive/MyDrive/t_brain/public_training_data
!chmod +x /content/drive/MyDrive/yolo_file/darknet-master/darknet
!cd /content/drive/MyDrive/yolo_file/darknet-master; ./darknet detector train cfg_steel/obj.data cfg_steel/yolov4_ad.cfg yolov4.conv.137 -dont_show -map -chart chart.png

In [ ]:
# # 若訓練中斷後; 接續訓練 (備用)

# !chmod +x /content/drive/MyDrive/t_brain/public_training_data
# !chmod +x /content/drive/MyDrive/yolo_file/darknet-master/darknet
# !cd /content/drive/MyDrive/yolo_file/darknet-master; ./darknet detector train cfg_steel/obj.data cfg_steel/yolov4_ad.cfg cfg_steel/weights/yolov4_ad_last.weights -dont_show -map -chart chart.png